<a href="https://colab.research.google.com/github/SaeidDaliriSusefi/ERA-5-Temperature-Downscaling/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap

In [2]:
ee.Authenticate()
ee.Initialize(project="Your google earth engine project name", opt_url='https://earthengine-highvolume.googleapis.com')

In [108]:
country_name =                                # Country's name            # str
region_name =                                 # Region's name             # str
sample_time_start =                           # Start data YYYY-MM-DD     # str
sample_time_end =                             # End data YYYY-MM-DD       # str
model_time_start =                            # End data YYYY-MM-DD       # str
model_time_end =                              # End data YYYY-MM-DD       # str

In [107]:
def convert_to_monthly_sum(collection, start, end):
    start_date = ee.Date(start)
    end_date = ee.Date(end)

    adjusted_end = ee.Algorithms.If(
        ee.Number(end_date.get('day')).eq(1),
        end_date.advance(-1, 'day'),
        end_date
    )
    end_date = ee.Date(adjusted_end)

    months = ee.List.sequence(0, end_date.difference(start_date, 'month'))

    def map_month(m):
        m = ee.Number(m)
        start_m = start_date.advance(m, 'month')
        end_m = start_m.advance(1, 'month')

        filtered = collection.filterDate(start_m, end_m)
        monthly_image = filtered.sum() \
            .set('system:time_start', start_m.millis())

        first_image = ee.Image(filtered.first())
        monthly_image = monthly_image.copyProperties(first_image, first_image.propertyNames())

        return monthly_image

    return ee.ImageCollection.fromImages(months.map(map_month))


def convert_to_monthly_average(collection, start, end):
    start_date = ee.Date(start)
    end_date = ee.Date(end)

    adjusted_end = ee.Algorithms.If(
        ee.Number(end_date.get('day')).eq(1),
        end_date.advance(-1, 'day'),
        end_date
    )
    end_date = ee.Date(adjusted_end)

    months = ee.List.sequence(0, end_date.difference(start_date, 'month'))

    def map_month(m):
        m = ee.Number(m)
        start_m = start_date.advance(m, 'month')
        end_m = start_m.advance(1, 'month')

        filtered = collection.filterDate(start_m, end_m)
        monthly_image = filtered.mean() \
            .set('system:time_start', start_m.millis())

        first_image = ee.Image(filtered.first())
        monthly_image = monthly_image.copyProperties(first_image, first_image.propertyNames())

        return monthly_image

    return ee.ImageCollection.fromImages(months.map(map_month))

In [111]:
country = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1").filter(ee.Filter.eq('ADM0_NAME',country_name))
Region = country.filter(ee.Filter.eq('ADM1_NAME',region_name ))
roi=Region.geometry()

In [122]:
Era5_11km = (
    ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
    .select('skin_temperature')
    .filterDate(sample_time_start, sample_time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)


In [122]:
Modis = (
    ee.ImageCollection('MODIS/061/MOD11A1')
    .select('LST_Day_1km')
    .filterDate(sample_time_start, sample_time_end)
    .map(lambda img: img.toInt()
         .copyProperties(img, img.propertyNames())
         .set('date', img.date().format('YYYY-MM')))
)

In [122]:
def merge_collections(era5_img):
    date = era5_img.get('date')  # Get the date from the ERA5 image

    modis_match = Modis.filter(ee.Filter.eq('date', date)).first()

    modis_lst = modis_match.select('LST_Day_1km')

    cons = ee.Image.constant(1)

    merged_image = era5_img.addBands(modis_lst.rename('MODIS_LST_Day_1km')).addBands(cons.rename('constant'))

    return merged_image

collection = Era5_11km.map(merge_collections)



In [117]:
model = collection.reduce(ee.Reducer.linearRegression(2, 1))
gain = model.select('coefficients').arrayGet([0,0])
offset = model.select('coefficients').arrayGet([1,0])

In [122]:
Map = geemap.Map(basemap="SATELLITE")

era5_1km = (
    ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
    .select('skin_temperature')
    .filterDate(model_time_start, model_time_end)
    .map(lambda img: (
        img.reproject(crs='EPSG:4326', scale=1000)
           .multiply(gain)  #
           .add(offset)
           .rename('predicted')
           .addBands(
               img.reproject(crs='EPSG:4326', scale=1000)
               .rename('era5_skin_temperature')
           )
           .addBands(ee.Image.constant(1).rename('constant'))
           .copyProperties(img, img.propertyNames())
    ))
)

count_1km = era5_1km.size().getInfo()
image_list_1km = era5_1km.toList(count_1km)

for i in range(count_1km):
    img = ee.Image(image_list_1km.get(i)).clip(roi)
    date = img.get('date').getInfo()

    stats_predicted = img.select('predicted').reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=roi,
        scale=1000,  #
        maxPixels=1e8
    ).getInfo()

    stats_era5 = img.select('era5_skin_temperature').reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=roi,
        scale=1000,
        maxPixels=1e8
    ).getInfo()

    min_predicted = stats_predicted['predicted_min']
    max_predicted = stats_predicted['predicted_max']

    min_era5 = stats_era5['era5_skin_temperature_min']
    max_era5 = stats_era5['era5_skin_temperature_max']

    vis_params_predicted = {
        'min': min_predicted,
        'max': max_predicted,
        'palette': ['Black', 'white']
    }

    vis_params_era5 = {
        'min': min_era5,
        'max': max_era5,
        'palette': ['Black', 'white']
    }

    Map.addLayer(img.select('predicted'), vis_params_predicted, f"Predicted ERA5 Skin Temperature {date}")
    Map.addLayer(img.select('era5_skin_temperature'), vis_params_era5, f"ERA5 Skin Temperature {date}")

Map.centerObject(roi, 5)

Map
